In [1]:
import wikipedia
from bs4 import BeautifulSoup as bs
import pandas as pd
import wikipediaapi
wiki = wikipediaapi.Wikipedia('en')
import pickle

In [ ]:
input_df = pd.read_csv('wiki_titles_output.csv')
input_df.pop('0')
MAIN = list(input_df['Template Numerical Toolkit'])

In [ ]:
def parse_kw(title):
    try:
        curpage = wikipedia.page(title).html()
        curpage = curpage[curpage.find('<div class="mw-parser-output"'):]
        curpage = curpage[curpage.find('<p>'):curpage.find('id="See_also"')]

        terms = []

        curpage = bs(curpage,'html.parser')
        for i in curpage.find_all('a'):
            if i['href'].find('/wiki/')!=-1 and i.text!='':
                terms.append(i.text.strip())

        return terms
    except:
        return

In [ ]:
def parse_text(title):
    try:
        page = wikipedia.page(title).content
        page = page.replace('\n','').replace('  ',' ').replace('==',' ')
        return page
    except:
        return

In [ ]:
FINAL = []
count = 0
save_num = 0
for i in MAIN[2000:]:
    FINAL.append(
    {
        'title' : i,
        'text' : parse_text(i),
        'kw' : parse_kw(i)
    }
    )
    count += 1
    print(int(count/len(MAIN)*100), '% <--->', str(count)+'/'+str(len(MAIN)),  end='\r')
    
    if(count % 1000 == 0): #each saved file contains 1000 wiki page objects
        save_num += 1
        with open('wiki_objects/wiki_page_objects_{}.pkl'.format(save_num), 'wb') as f:
            pickle.dump(FINAL, f)
        FINAL = []